In [1]:
import sys
import pandas as pd
import numpy as np

In [3]:
path = "D:/Vaastu/Data/VASTU_SCORE_CARD_111122_2_1_Output/"
path = "D:/Vaastu/Data/VASTU_SCORE_CARD_111122_2_1_Retro_Output/VASTU_SCORE_CARD_111122_2_1_Retro_Output/"
filename = "Account.csv"

df =  pd.read_csv(path+filename, sep = "|",error_bad_lines=False)

colnames = df.columns
newcolnames = []
for col in colnames:
    newcol = col.replace(" ", "_")
    newcol = newcol.replace("-", "_")
    newcol = newcol.replace("/", "_")
    newcol = newcol.replace(":", "_")
    newcol = newcol.replace("__", "_")
    newcolnames.append(newcol)
    
    
df.columns = newcolnames
from datetime import date
df['DISBURSED_DT'] = pd.to_datetime(df['DISBURSED_DT'],format="%d-%m-%Y",errors='coerce')
df['DATE_REPORTED'] = pd.to_datetime(df['DATE_REPORTED'],format="%d-%m-%Y",errors='coerce')

df.shape, df.head()

C:\Users\romil\AppData\Local\Temp\ipykernel_7680\2546687590.py:5: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df =  pd.read_csv(path+filename, sep = "|",error_bad_lines=False)
C:\Users\romil\AppData\Local\Temp\ipykernel_7680\2546687590.py:5: DtypeWarning: Columns (4,23) have mixed types. Specify dtype option on import or set low_memory=False.
  df =  pd.read_csv(path+filename, sep = "|",error_bad_lines=False)


((1554704, 42),
                   CREDT_RPT_ID  LOS_APP_ID  CANDIDATE__ID  CUSTOMER_ID_MBR_ID  \
 0  VAST221119CR130117923734081  N004500061      208985439                 NaN   
 1  VAST221119CR130117923734081  N004500061      140942914                 NaN   
 2  VAST221119CR130117923734081  N004500061      159060193                 NaN   
 3  VAST221119CR130117923734081  N004500061      812520964                 NaN   
 4  VAST221119CR130117923734081  N004500061      559060193                 NaN   
 
   BRANCH KENDRA  SELF_INDICATOR MATCH_TYPE ACC_NUM CREDIT_GRANTOR  ...  \
 0    NaN    NaN           False    PRIMARY    XXXX           XXXX  ...   
 1    NaN    NaN           False    PRIMARY    XXXX           XXXX  ...   
 2    NaN    NaN           False    PRIMARY    XXXX           XXXX  ...   
 3    NaN    NaN           False    PRIMARY    XXXX           XXXX  ...   
 4    NaN    NaN           False    PRIMARY    XXXX           XXXX  ...   
 
                                      

In [4]:
df.to_pickle("df_retro_entire.pkl")


In [2]:
#Import base with CV tradelines having atleast 18MOB of data
df_cv= pd.read_pickle("df_cv_filtered.pkl")
df_cv['DISBURSED_DT_YRMTH']=df_cv['DISBURSED_DT'].dt.strftime('%Y-%m')
df_cv['DATE_REPORTED_YRMTH']=df_cv['DATE_REPORTED'].dt.strftime('%Y-%m')

df_cv['DISBURSED_DT_YR']=df_cv['DISBURSED_DT'].dt.strftime('%Y')
df_cv['DATE_REPORTED_YR']=df_cv['DATE_REPORTED'].dt.strftime('%Y')

#Include only NBF Loans, Individual,  Disbursed Years : 2015 - 2019


cond1 = (df_cv.OWNERSHIP_IND == 'Individual')
cond2 = (df_cv.CONTRIBUTOR_TYPE == 'NBF')
cond3 = (df_cv.DISBURSED_DT_YR.isin(['2015','2016','2017','2018','2019']))

df_cv_filtered = df_cv[cond1 & cond2 & cond3 ]

df_cv.shape, df_cv['LOS_APP_ID'].nunique() , df_cv_filtered.shape, df_cv_filtered['LOS_APP_ID'].nunique()




((525133, 48), 75753, (85950, 48), 36886)

In [3]:
df_filtered = df.merge(pd.DataFrame(df_cv_filtered.LOS_APP_ID.unique() , columns = ['LOS_APP_ID'] ) , on = 'LOS_APP_ID', how='inner' )
df_filtered.to_pickle("df_retro_filtered.pkl")

df.shape, df['LOS_APP_ID'].nunique() , df_filtered.shape, df_filtered['LOS_APP_ID'].nunique()


NameError: name 'df' is not defined

In [ ]:

df_cv_filtered_v2 = df_cv_filtered.merge(pd.DataFrame(df_filtered.LOS_APP_ID.unique() , columns = ['LOS_APP_ID'] ) , on = 'LOS_APP_ID', how='inner' )

df_cv_filtered_v2.to_pickle("df_goodbad_filtered.pkl")
df_cv_filtered.shape, df_cv_filtered['CANDIDATE__ID'].nunique() , df_cv_filtered_v2.shape, df_cv_filtered_v2['LOS_APP_ID'].nunique()



In [4]:
## Reading processed datasets
df_cv_goodbad= pd.read_pickle("df_goodbad_filtered.pkl")
df_cv_retro = pd.read_pickle("df_retro_filtered.pkl")

df_cv_goodbad.shape, df_cv_retro.shape

((83364, 48), (869346, 42))

In [5]:


firstdate_curr_scrub = df_cv_goodbad.groupby('LOS_APP_ID').agg({'DISBURSED_DT' : min}).reset_index()
firstdate_curr_scrub.rename(columns = {'DISBURSED_DT':'DISBURSED_DT_curr_scrub'}, inplace = True)

first_date_retro_scrub = df_cv_retro.groupby('LOS_APP_ID').agg({'DISBURSED_DT' : min}).reset_index()
first_date_retro_scrub.rename(columns = {'DISBURSED_DT':'DISBURSED_DT_retro_scrub'}, inplace = True)


df_cv_retro_map = first_date_retro_scrub.merge(firstdate_curr_scrub, how="left" , on ="LOS_APP_ID")
from datetime import date
#df_cv_retro_map['DISBURSED_DT_curr_scrub'] = pd.to_datetime(df_cv_retro_map['DISBURSED_DT'],format="%d-%m-%Y",errors='coerce')
#df_cv_retro_map['DATE_REPORTED'] = pd.to_datetime(df_cv_retro_map['DATE_REPORTED'],format="%d-%m-%Y",errors='coerce')
#df_cv_retro_map['DISBURSED_DT_goodbad'] = pd.to_datetime(df_cv_retro_map['DISBURSED_DT_goodbad'],format="%d-%m-%Y",errors='coerce')


df_cv_retro_map.shape, df_cv_retro_map[df_cv_retro_map['DISBURSED_DT_retro_scrub'] < df_cv_retro_map['DISBURSED_DT_curr_scrub']].shape



((34718, 3), (33679, 3))

In [6]:
cols = ['Loan_id',"DISBURSED_DT","DATE_REPORTED" ,'LOS_APP_ID' , "ACCOUNT_STATUS",
        "REPORTED_DATE__HIST_","DPD__HIST",
       "AMT_OVERDUE__HIST_","CUR_BAL__HIST_" ]
df_cv_goodbad['Loan_id'] = df_cv_goodbad.reset_index().index

temp = df_cv_goodbad[cols]
#temp['DPD__HIST'] = temp['DPD__HIST'].astype(str) 
for n in range(37):
    print(n+1)
    st= n*3
    lst =(n+1)*3
    #print(st,lst)
    temp['DPD' + str(n+1)]= temp.DPD__HIST.str.slice(st,lst)
    st= n*8 + n
    lst =(n+1)*8 + n
    #print(st,lst)
    temp['Reported_Dt' + str(n+1)]= temp['REPORTED_DATE__HIST_'].str.slice(st,lst)
    temp['Reported_Dt' + str(n+1)] = pd.to_datetime(temp['Reported_Dt' + str(n+1)],format="%Y%m%d",errors='coerce')



print(temp.shape)

temp[['REPORTED_DATE__HIST_', 'Reported_Dt1','Reported_Dt2']].head(2)

1
2
3
4


C:\Users\romil\AppData\Local\Temp\ipykernel_8984\1770647874.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp['DPD' + str(n+1)]= temp.DPD__HIST.str.slice(st,lst)
C:\Users\romil\AppData\Local\Temp\ipykernel_8984\1770647874.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp['Reported_Dt' + str(n+1)]= temp['REPORTED_DATE__HIST_'].str.slice(st,lst)
C:\Users\romil\AppData\Local\Temp\ipykernel_8984\1770647874.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a Da

5
6
7
8


C:\Users\romil\AppData\Local\Temp\ipykernel_8984\1770647874.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp['DPD' + str(n+1)]= temp.DPD__HIST.str.slice(st,lst)
C:\Users\romil\AppData\Local\Temp\ipykernel_8984\1770647874.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp['Reported_Dt' + str(n+1)]= temp['REPORTED_DATE__HIST_'].str.slice(st,lst)
C:\Users\romil\AppData\Local\Temp\ipykernel_8984\1770647874.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a Da

9
10
11


...

C:\Users\romil\AppData\Local\Temp\ipykernel_8984\1770647874.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp['DPD' + str(n+1)]= temp.DPD__HIST.str.slice(st,lst)
C:\Users\romil\AppData\Local\Temp\ipykernel_8984\1770647874.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp['Reported_Dt' + str(n+1)]= temp['REPORTED_DATE__HIST_'].str.slice(st,lst)
C:\Users\romil\AppData\Local\Temp\ipykernel_8984\1770647874.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a Da

16
17
18
19


C:\Users\romil\AppData\Local\Temp\ipykernel_8984\1770647874.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp['DPD' + str(n+1)]= temp.DPD__HIST.str.slice(st,lst)
C:\Users\romil\AppData\Local\Temp\ipykernel_8984\1770647874.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp['Reported_Dt' + str(n+1)]= temp['REPORTED_DATE__HIST_'].str.slice(st,lst)
C:\Users\romil\AppData\Local\Temp\ipykernel_8984\1770647874.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a Da

20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
(83364, 83)


,REPORTED_DATE__HIST_,Reported_Dt1,Reported_Dt2
0,"20180831,20180731,20180630,20180531,20180430,2...",2018-08-31,2018-07-31
1,"20180831,20180731,20180630,20180531,20180430,2...",2018-08-31,2018-07-31


In [7]:
met = 'AMT_OVERDUE'
col ="AMT_OVERDUE__HIST_"

names_arr = []
for n in range(1,38):
    names_arr.append(met + str(n))
    
print(temp.shape)
temp[names_arr] = temp[col].str.split(',', expand=True)

met = 'CUR_BAL'
col ="CUR_BAL__HIST_"

names_arr = []
for n in range(1,38):
    names_arr.append(met + str(n))
    
print(temp.shape)
temp[names_arr] = temp[col].str.split(',', expand=True)



temp[['CUR_BAL__HIST_','CUR_BAL1','CUR_BAL2','CUR_BAL3']].head(4)

(83364, 83)
(83364, 120)


,CUR_BAL__HIST_,CUR_BAL1,CUR_BAL2,CUR_BAL3
0,"0,0,14570,29440,44310,59180,74050,88920,103790...",0,0,14570
1,"0,116777,154755,192265,229311,265901,302038,33...",0,116777,154755
2,"0,28742,42834,56744,70474,,97401,110603,123634...",0,28742,42834
3,"0,13243,26294,39152,51820,64300,76596,88710,10...",0,13243,26294


In [9]:
df_long=pd.wide_to_long(temp, 
                stubnames=['CUR_BAL','Reported_Dt','DPD','AMT_OVERDUE'], 
                i=['Loan_id',"DISBURSED_DT","DATE_REPORTED" ,'LOS_APP_ID' , "ACCOUNT_STATUS"], 
                j='Month').reset_index()

df_long['MOB'] = ((df_long.Reported_Dt - pd.to_datetime(df_long.DISBURSED_DT) )/np.timedelta64(1, 'M'))

df_long['DPD']=pd.to_numeric(df_long['DPD'], errors = 'coerce')
df_long['AMT_OVERDUE']=pd.to_numeric(df_long['AMT_OVERDUE'], errors = 'coerce')
df_long['AMT_OVERDUE']=pd.to_numeric(df_long['AMT_OVERDUE'], errors = 'coerce')
df_long.head()

,Loan_id,DISBURSED_DT,DATE_REPORTED,LOS_APP_ID,ACCOUNT_STATUS,Month,CUR_BAL__HIST_,DPD__HIST,AMT_OVERDUE__HIST_,REPORTED_DATE__HIST_,CUR_BAL,Reported_Dt,DPD,AMT_OVERDUE,MOB
0,0,2016-07-27,2018-08-31,L022500180,Closed,1,"0,0,14570,29440,44310,59180,74050,88920,103790...",0000000000000000000000000000000000000000000000...,",,,,,,,,,,,,,,,,,,,,,,,,,,","20180831,20180731,20180630,20180531,20180430,2...",0,2018-08-31,0.0,NaN,25.133986
1,0,2016-07-27,2018-08-31,L022500180,Closed,2,"0,0,14570,29440,44310,59180,74050,88920,103790...",0000000000000000000000000000000000000000000000...,",,,,,,,,,,,,,,,,,,,,,,,,,,","20180831,20180731,20180630,20180531,20180430,2...",0,2018-07-31,0.0,NaN,24.115485
2,0,2016-07-27,2018-08-31,L022500180,Closed,3,"0,0,14570,29440,44310,59180,74050,88920,103790...",0000000000000000000000000000000000000000000000...,",,,,,,,,,,,,,,,,,,,,,,,,,,","20180831,20180731,20180630,20180531,20180430,2...",14570,2018-06-30,0.0,NaN,23.096984
3,0,2016-07-27,2018-08-31,L022500180,Closed,4,"0,0,14570,29440,44310,59180,74050,88920,103790...",0000000000000000000000000000000000000000000000...,",,,,,,,,,,,,,,,,,,,,,,,,,,","20180831,20180731,20180630,20180531,20180430,2...",29440,2018-05-31,0.0,NaN,22.111337
4,0,2016-07-27,2018-08-31,L022500180,Closed,5,"0,0,14570,29440,44310,59180,74050,88920,103790...",0000000000000000000000000000000000000000000000...,",,,,,,,,,,,,,,,,,,,,,,,,,,","20180831,20180731,20180630,20180531,20180430,2...",44310,2018-04-30,0.0,NaN,21.092836


In [10]:
#18 months
loan_agg = df_long[df_long['MOB'] <= 18].groupby('Loan_id').agg({'DPD' : max}).reset_index()
loan_agg['DPD_1_plus_18'] = np.where(loan_agg['DPD'] > 0 ,1,0)
loan_agg['DPD_30_plus_18'] = np.where(loan_agg['DPD'] > 30 ,1,0)
loan_agg['DPD_60_plus_18'] = np.where(loan_agg['DPD'] > 60 ,1,0)

loan_agg.shape,loan_agg.DPD_1_plus_18.mean(),loan_agg.DPD_30_plus_18.mean(),loan_agg.DPD_60_plus_18.mean()

((81564, 5), 0.4365651512922368, 0.2146044823696729, 0.1159089794517189)

In [14]:
cus_agg = df_long[df_long['MOB'] <= 18].groupby('LOS_APP_ID').agg({'DPD' : max}).reset_index()
cus_agg['DPD_1_plus_18'] = np.where(cus_agg['DPD'] > 0 ,1,0)
cus_agg['DPD_30_plus_18'] = np.where(cus_agg['DPD'] > 30 ,1,0)
cus_agg['DPD_60_plus_18'] = np.where(cus_agg['DPD'] > 60 ,1,0)
cus_agg.to_pickle('driverset_18M.pkl')
cus_agg.shape,cus_agg.DPD_1_plus_18.mean(),cus_agg.DPD_30_plus_18.mean(),cus_agg.DPD_60_plus_18.mean()

((34305, 5), 0.5323422241655735, 0.2979449059903804, 0.17023757469756595)

In [10]:
#12 months
loan_agg = df_long[df_long['MOB'] <= 12].groupby('Loan_id').agg({'DPD' : max}).reset_index()
loan_agg['DPD_1_plus_12'] = np.where(loan_agg['DPD'] > 0 ,1,0)
loan_agg['DPD_30_plus_12'] = np.where(loan_agg['DPD'] > 30 ,1,0)
loan_agg['DPD_60_plus_12'] = np.where(loan_agg['DPD'] > 60 ,1,0)

loan_agg.shape,loan_agg.DPD_1_plus_12.mean(),loan_agg.DPD_30_plus_12.mean(),loan_agg.DPD_60_plus_12.mean()

((80930, 5), 0.3150006178178673, 0.12167305078462869, 0.053305325590016064)

In [12]:
cus_agg2 = df_long[df_long['MOB'] <= 12].groupby('LOS_APP_ID').agg({'DPD' : max}).reset_index()
cus_agg2['DPD_1_plus_12'] = np.where(cus_agg2['DPD'] > 0 ,1,0)
cus_agg2['DPD_30_plus_12'] = np.where(cus_agg2['DPD'] > 30 ,1,0)
cus_agg2['DPD_60_plus_12'] = np.where(cus_agg2['DPD'] > 60 ,1,0)

cus_agg2.shape,cus_agg2.DPD_1_plus_12.mean(),cus_agg2.DPD_30_plus_12.mean(),cus_agg.DPD_60_plus_12.mean()

((34140, 5), 0.4131517281780902, 0.18588166373755127, 0.08728763913298183)

In [ ]:
cus_agg_vf= cus_agg

In [36]:

df_cv_retro = pd.read_pickle("df_retro_filtered.pkl")

firstdate_curr_scrub = df_cv_goodbad.groupby('LOS_APP_ID').agg({'DISBURSED_DT' : min}).reset_index()
firstdate_curr_scrub.rename(columns = {'DISBURSED_DT':'DISBURSED_DT_curr_scrub'}, inplace = True)

df_cv_retro= df_cv_retro.merge(firstdate_curr_scrub, how="inner" , on ="LOS_APP_ID")


from datetime import date
df_cv_retro['DISBURSED_DT'] = pd.to_datetime(df_cv_retro['DISBURSED_DT'],format="%d-%m-%Y",errors='coerce')
df_cv_retro['DATE_REPORTED'] = pd.to_datetime(df_cv_retro['DATE_REPORTED'],format="%d-%m-%Y",errors='coerce')
df_cv_retro['DISBURSED_DT_curr_scrub'] = pd.to_datetime(df_cv_retro['DISBURSED_DT_curr_scrub'],format="%d-%m-%Y",errors='coerce')
df_cv_retro_useful = df_cv_retro[df_cv_retro['DISBURSED_DT'] < df_cv_retro['DISBURSED_DT_curr_scrub'] ]

df_cv_retro.shape,df_cv_retro_useful.shape



((869346, 43), (734211, 43))

In [37]:
cols = ['Loan_id',"DISBURSED_DT","DATE_REPORTED" ,'LOS_APP_ID' , "ACCOUNT_STATUS", "REPORTED_DATE__HIST_","DPD__HIST",
       "AMT_OVERDUE__HIST_","CUR_BAL__HIST_"]
df_cv_retro_useful['Loan_id'] = df_cv_retro_useful.reset_index().index

temp = df_cv_retro_useful[cols]
#temp['DPD__HIST'] = temp['DPD__HIST'].astype(str) 
for n in range(37):
    print(n+1)
    st= n*3
    lst =(n+1)*3
    #print(st,lst)
    temp['DPD' + str(n+1)]= temp.DPD__HIST.str.slice(st,lst)
    st= n*8 + n
    lst =(n+1)*8 + n
    #print(st,lst)
    temp['Reported_Dt' + str(n+1)]= temp['REPORTED_DATE__HIST_'].str.slice(st,lst)
    temp['Reported_Dt' + str(n+1)] = pd.to_datetime(temp['Reported_Dt' + str(n+1)],format="%Y%m%d",errors='coerce')


met = 'AMT_OVERDUE'
col ="AMT_OVERDUE__HIST_"

names_arr = []
for n in range(1,38):
    names_arr.append(met + str(n))
    
print(temp.shape)
temp[names_arr] = temp[col].str.split(',', expand=True)

met = 'CUR_BAL'
col ="CUR_BAL__HIST_"

names_arr = []
for n in range(1,38):
    names_arr.append(met + str(n))
    
print(temp.shape)
temp[names_arr] = temp[col].str.split(',', expand=True)

print(temp[['CUR_BAL__HIST_','CUR_BAL1','CUR_BAL2','CUR_BAL3']].head(4))

print(temp[['REPORTED_DATE__HIST_', 'Reported_Dt1','Reported_Dt2']].head(2))

C:\Users\romil\AppData\Local\Temp\ipykernel_8840\702606262.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cv_retro_useful['Loan_id'] = df_cv_retro_useful.reset_index().index


1


C:\Users\romil\AppData\Local\Temp\ipykernel_8840\702606262.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp['DPD' + str(n+1)]= temp.DPD__HIST.str.slice(st,lst)
C:\Users\romil\AppData\Local\Temp\ipykernel_8840\702606262.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp['Reported_Dt' + str(n+1)]= temp['REPORTED_DATE__HIST_'].str.slice(st,lst)
C:\Users\romil\AppData\Local\Temp\ipykernel_8840\702606262.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataF

2


C:\Users\romil\AppData\Local\Temp\ipykernel_8840\702606262.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp['DPD' + str(n+1)]= temp.DPD__HIST.str.slice(st,lst)
C:\Users\romil\AppData\Local\Temp\ipykernel_8840\702606262.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp['Reported_Dt' + str(n+1)]= temp['REPORTED_DATE__HIST_'].str.slice(st,lst)
C:\Users\romil\AppData\Local\Temp\ipykernel_8840\702606262.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataF

...

18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
(734211, 83)


C:\Users\romil\AppData\Local\Temp\ipykernel_8840\702606262.py:28: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  temp[names_arr] = temp[col].str.split(',', expand=True)
C:\Users\romil\AppData\Local\Temp\ipykernel_8840\702606262.py:28: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  temp[names_arr] = temp[col].str.split(',', expand=True)
C:\Users\romil\AppData\Local\Temp\ipykernel_8840\702606262.py:28: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor

(734211, 120)


C:\Users\romil\AppData\Local\Temp\ipykernel_8840\702606262.py:38: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  temp[names_arr] = temp[col].str.split(',', expand=True)
C:\Users\romil\AppData\Local\Temp\ipykernel_8840\702606262.py:38: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  temp[names_arr] = temp[col].str.split(',', expand=True)
C:\Users\romil\AppData\Local\Temp\ipykernel_8840\702606262.py:38: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor

                                      CUR_BAL__HIST_ CUR_BAL1 CUR_BAL2  \
1               0,32221,358221,396181,415161,434141,        0    32221   
2  0,137284,908284,942784,977264,1011764,1046264,...        0   137284   
4  0,100180,668630,694130,719610,745110,770610,79...        0   100180   
5                       0,0,23540,47120,70720,94320,        0        0   

  CUR_BAL3  
1   358221  
2   908284  
4   668630  
5    23540  
                                REPORTED_DATE__HIST_ Reported_Dt1 Reported_Dt2
1  20141231,20141130,20141031,20140930,20140831,2...   2014-12-31   2014-11-30
2  20150228,20150131,20141231,20141130,20141031,2...   2015-02-28   2015-01-31


In [38]:
temp.shape, temp.LOS_APP_ID.nunique(), temp.Loan_id.nunique()

((734211, 157), 33679, 734211)

In [ ]:
temp

In [13]:
df_cv_retro_map.columns

Index(['CREDT_RPT_ID', 'LOS_APP_ID', 'CANDIDATE__ID', 'CUSTOMER_ID_MBR_ID',
       'BRANCH', 'KENDRA', 'SELF_INDICATOR', 'MATCH_TYPE', 'ACC_NUM',
       'CREDIT_GRANTOR', 'ACCT_TYPE', 'CONTRIBUTOR_TYPE', 'DATE_REPORTED',
       'OWNERSHIP_IND', 'ACCOUNT_STATUS', 'DISBURSED_DT', 'CLOSE_DT',
       'LAST_PAYMENT_DATE', 'CREDIT_LIMIT_SANC_AMT',
       '_DISBURSED_AMT_HIGH_CREDIT', '_INSTALLMENT_AMT', '_CURRENT_BAL',
       'INSTALLMENT_FREQUENCY', 'WRITE_OFF_DATE', '_OVERDUE_AMT',
       '_WRITE_OFF_AMT', 'ASSET_CLASS', '_ACCOUNT_REMARKS', 'LINKED_ACCOUNTS',
       'REPORTED_DATE__HIST_', 'DPD__HIST', 'ASSET_CLASS__HIST_',
       'HIGH_CRD__HIST_', 'CUR_BAL__HIST_', 'DAS__HIST_', 'AMT_OVERDUE__HIST_',
       'AMT_PAID__HIST_', 'INCOME_', '_INCOME_INDICATOR_', 'TENURE_',
       '_OCCUPATION', 'Unnamed_41', 'DISBURSED_DT_goodbad'],
      dtype='object')

In [75]:
import datetime
df1['DISBURSED_DT_YRMTH']=df1['DISBURSED_DT'].dt.strftime('%Y-%m')
df1['DATE_REPORTED_YRMTH']=df1['DATE_REPORTED'].dt.strftime('%Y-%m')

df1['DISBURSED_DT_YR']=df1['DISBURSED_DT'].dt.strftime('%Y')
df1['DATE_REPORTED_YR']=df1['DATE_REPORTED'].dt.strftime('%Y')

<ipython-input-75-fb981f55ecd4>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['DISBURSED_DT_YRMTH']=df1['DISBURSED_DT'].dt.strftime('%Y-%m')
<ipython-input-75-fb981f55ecd4>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['DATE_REPORTED_YRMTH']=df1['DATE_REPORTED'].dt.strftime('%Y-%m')
<ipython-input-75-fb981f55ecd4>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documen

In [79]:
print(df1.shape, df1[df1['Month_diff_betweenDisb_reported'] <= 36 ].shape)
df1['Monthdiff_less_than36_atleast18m'] = np.where((df1['Month_diff_betweenDisb_reported'] <=36) & (df1['Month_diff_betweenDisb_reported'] >=18), 1 , 0 )
df1.groupby(['DISBURSED_DT_YR'])['Monthdiff_less_than36_atleast18m'].agg(['sum', 'count'])


(1634489, 48) (780113, 48)


<ipython-input-79-d8d13042102a>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['Monthdiff_less_than36_atleast18m'] = np.where((df1['Month_diff_betweenDisb_reported'] <=36) & (df1['Month_diff_betweenDisb_reported'] >=18), 1 , 0 )


,sum,count
DISBURSED_DT_YR,,
1992,0,8
1994,0,3
1995,0,3
1996,0,6
1997,0,6
1998,0,5
1999,0,21
2000,0,38
2001,0,83


In [80]:
df_cv= df1[df1['Monthdiff_less_than36_atleast18m']==1]
df_cv.shape, df1.shape

((525133, 48), (1634489, 48))

In [81]:
del df,df1

In [85]:
df_cv.head()

,CREDT_RPT_ID,LOS_APP_ID,CANDIDATE__ID,CUSTOMER_ID_MBR_ID,BRANCH,KENDRA,SELF_INDICATOR,MATCH_TYPE,ACC_NUM,CREDIT_GRANTOR,...,_INCOME_INDICATOR_,TENURE_,_OCCUPATION,Unnamed_41,Month_diff_betweenDisb_reported,DISBURSED_DT_YRMTH,DATE_REPORTED_YRMTH,DISBURSED_DT_YR,DATE_REPORTED_YR,Monthdiff_less_than36_atleast18m
2,VAST221114CR257974523894971,L022500180,253452087,NaN,NaN,NaN,False,PRIMARY,XXXX,XXXX,...,NaN,23.0,NaN,NaN,25.133986,2016-07,2018-08,2016,2018,1
7,VAST221114CR85724523894971,401300310033285,802673876,NaN,NaN,NaN,False,PRIMARY,XXXX,XXXX,...,M,NaN,SELF EMPLOYED,NaN,35.220436,2015-11,2018-10,2015,2018,1
17,VAST221114CR886434523894971,3389357,4159712081,NaN,NaN,NaN,False,PRIMARY,XXXX,XXXX,...,NaN,30.0,OTHERS,NaN,35.023306,2017-11,2020-10,2017,2020,1
22,VAST221114CR456354523894971,137100310054159,270519689,NaN,NaN,NaN,False,PRIMARY,XXXX,XXXX,...,NaN,24.0,NaN,NaN,23.688371,2017-02,2019-01,2017,2019,1
26,VAST221114CR456354523894971,137100310054159,4969194642,NaN,NaN,NaN,False,PRIMARY,XXXX,XXXX,...,M,NaN,NaN,NaN,29.043718,2019-11,2022-04,2019,2022,1


In [82]:
!pip install pandas-profiling

^C


In [86]:
df_cv.to_pickle("df_cv_filtered.pkl")

In [3]:
df_cv= pd.read_pickle("df_cv_filtered.pkl")

In [4]:
df_cv.head()

,CREDT_RPT_ID,LOS_APP_ID,CANDIDATE__ID,CUSTOMER_ID_MBR_ID,BRANCH,KENDRA,SELF_INDICATOR,MATCH_TYPE,ACC_NUM,CREDIT_GRANTOR,...,_INCOME_INDICATOR_,TENURE_,_OCCUPATION,Unnamed_41,Month_diff_betweenDisb_reported,DISBURSED_DT_YRMTH,DATE_REPORTED_YRMTH,DISBURSED_DT_YR,DATE_REPORTED_YR,Monthdiff_less_than36_atleast18m
2,VAST221114CR257974523894971,L022500180,253452087,NaN,NaN,NaN,False,PRIMARY,XXXX,XXXX,...,NaN,23.0,NaN,NaN,25.133986,2016-07,2018-08,2016,2018,1
7,VAST221114CR85724523894971,401300310033285,802673876,NaN,NaN,NaN,False,PRIMARY,XXXX,XXXX,...,M,NaN,SELF EMPLOYED,NaN,35.220436,2015-11,2018-10,2015,2018,1
17,VAST221114CR886434523894971,3389357,4159712081,NaN,NaN,NaN,False,PRIMARY,XXXX,XXXX,...,NaN,30.0,OTHERS,NaN,35.023306,2017-11,2020-10,2017,2020,1
22,VAST221114CR456354523894971,137100310054159,270519689,NaN,NaN,NaN,False,PRIMARY,XXXX,XXXX,...,NaN,24.0,NaN,NaN,23.688371,2017-02,2019-01,2017,2019,1
26,VAST221114CR456354523894971,137100310054159,4969194642,NaN,NaN,NaN,False,PRIMARY,XXXX,XXXX,...,M,NaN,NaN,NaN,29.043718,2019-11,2022-04,2019,2022,1


In [5]:
import sys
!{sys.executable} -m pip install pandas-profiling

  Using cached pandas_profiling-3.5.0-py2.py3-none-any.whl (325 kB)
     -------------------------------------- 685.8/685.8 kB 8.7 MB/s eta 0:00:00
     ---------------------------------------- 2.1/2.1 MB 5.2 MB/s eta 0:00:00
  Using cached typeguard-2.13.3-py3-none-any.whl (17 kB)
  Using cached htmlmin-0.1.12.tar.gz (19 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached multimethod-1.9-py3-none-any.whl (10 kB)
  Using cached visions-0.7.5-py3-none-any.whl (102 kB)
  Using cached tangled_up_in_unicode-0.2.0-py3-none-any.whl (4.7 MB)
  Using cached ImageHash-4.3.1-py2.py3-none-any.whl (296 kB)
  Created wheel for htmlmin: filename=htmlmin-0.1.12-py3-none-any.whl size=27082 sha256=52132bde09bed01d5491588bd37aa37ad95519d44c13956b72df86e426917029
  Stored in directory: c:\users\romil\appdata\local\pip\cache\wheels\1d\05\04\c6d7d3b66539d9e659ac6dfe81e2d0fd4c1a8316cc5a403300
Successfully built htmlmin


In [7]:
import pandas_profiling as pp
  
# forming ProfileReport and save
# as output.html file
profile = pp.ProfileReport(df_cv)
profile.to_file("EDA_pp.html")

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

C:\Users\romil\Anaconda\lib\site-packages\scipy\stats\_stats_py.py:110: RuntimeWarning: The input array could not be properly checked for nan values. nan values will be ignored.
  warnings.warn("The input array could not be properly "
C:\Users\romil\Anaconda\lib\site-packages\scipy\stats\_stats_py.py:110: RuntimeWarning: The input array could not be properly checked for nan values. nan values will be ignored.
  warnings.warn("The input array could not be properly "
C:\Users\romil\Anaconda\lib\site-packages\scipy\stats\_stats_py.py:110: RuntimeWarning: The input array could not be properly checked for nan values. nan values will be ignored.
  warnings.warn("The input array could not be properly "
C:\Users\romil\Anaconda\lib\site-packages\scipy\stats\_stats_py.py:110: RuntimeWarning: The input array could not be properly checked for nan values. nan values will be ignored.
  warnings.warn("The input array could not be properly "
C:\Users\romil\Anaconda\lib\site-packages\scipy\stats\_stats

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

In [9]:
metrics = ['AMT_OVERDUE__HIST_','AMT_PAID__HIST_','CUR_BAL__HIST_']

df_cv['DISBURSED_DT'] = pd.to_datetime(df_cv['DISBURSED_DT'],format="%d-%m-%Y",errors='coerce')
df_cv['DATE_REPORTED'] = pd.to_datetime(df_cv['DATE_REPORTED'],format="%d-%m-%Y",errors='coerce')


temp = pd.concat([df_cv[['LOS_APP_ID','CANDIDATE__ID' , 'DISBURSED_DT', 'DATE_REPORTED',"AMT_OVERDUE__HIST_"]], df_cv["AMT_OVERDUE__HIST_"].str.split(',', expand=True)], axis=1)

translist = list(set(temp.columns) - set(['LOS_APP_ID','CANDIDATE__ID' , 'DISBURSED_DT', 'DATE_REPORTED',"AMT_OVERDUE__HIST_"]))

temp_l = pd.melt(temp, id_vars=['LOS_APP_ID','CANDIDATE__ID' , 'DISBURSED_DT', 'DATE_REPORTED'], value_vars=translist )
temp_l.rename(columns={'value': 'AMT_OVERDUE__HIST_'}, inplace=True)

temp_l['variable'] = temp_l['variable'].astype('int')
temp_l['variable'] = temp_l['variable'].fillna(36)



In [11]:
temp_l['Mont_diff'] = ((temp_l.DATE_REPORTED - pd.to_datetime(temp_l.DISBURSED_DT) )/np.timedelta64(1, 'M')) + temp_l['variable']
temp_l.head()

,LOS_APP_ID,CANDIDATE__ID,DISBURSED_DT,DATE_REPORTED,variable,AMT_OVERDUE__HIST_,Mont_diff
0,L022500180,253452087,2016-07-27,2018-08-31,0,,25.133986
1,401300310033285,802673876,2015-11-24,2018-10-31,0,0,35.220436
2,3389357,4159712081,2017-11-30,2020-10-31,0,0,35.023306
3,137100310054159,270519689,2017-02-09,2019-01-31,0,0,23.688371
4,137100310054159,4969194642,2019-11-28,2022-04-30,0,0,29.043718
